<a href="https://colab.research.google.com/github/miroorc/Capstone-Project/blob/main/YOSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade datasets

In [ ]:
pip install transformers

In [ ]:
pip install transformers[torch]

In [ ]:
#load the ecthra dataset from lex_glue
from datasets import load_dataset
ecthra = load_dataset("lex_glue", "ecthr_a")

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#ecthra['train']=ecthra['train'].select([10])
#ecthra['test']=ecthra['test'].select(range(10))
#ecthra['validation']=ecthra['validation'].select(range(10))

In [ ]:
#Extract the train, test, and validation parts of the ecthra dataset.
train=ecthra['train']
validation=ecthra['validation']
test=ecthra['test']

In [ ]:
#convert label to one-hot form
import numpy as np
num_classes = 10

def one_hot_encode_multi(labels, num_classes):
    encoded_labels = np.zeros((len(labels),num_classes))
    for i, sublist in enumerate(labels):
        encoded_labels[i, sublist] = 1
    return encoded_labels

In [ ]:
train_one_hot=one_hot_encode_multi(train['labels'],10)
train_one_hot = train_one_hot.tolist()
train=train.add_column(name="labels_one_hot", column=train_one_hot)

In [ ]:
test_one_hot=one_hot_encode_multi(test['labels'],10)
test_one_hot = test_one_hot.tolist()
test=test.add_column(name="labels_one_hot", column=test_one_hot)

In [ ]:
validation_one_hot=one_hot_encode_multi(validation['labels'],10)
validation_one_hot = validation_one_hot.tolist()
validation=validation.add_column(name="labels_one_hot", column=validation_one_hot)

In [ ]:
#clean the dataset to remove meaningless characters
import re
def format_text(text):
    text_format = []
    text=[[re.sub(r"^\d+\.\s*","",s) for s in sublst] for sublst in text]
    for each in text:
        text_format.append(''.join(each))
    return text_format

In [ ]:
train_text_format=format_text(train['text'])
train=train.remove_columns('text')
train=train.add_column(name="text", column=train_text_format)
ecthra['train']=train

In [ ]:
test_text_format=format_text(test['text'])
test=test.remove_columns('text')
test=test.add_column(name="text", column=test_text_format)
ecthra['test']=test

In [ ]:
validation_text_format=format_text(validation['text'])
validation=validation.remove_columns('text')
validation=validation.add_column(name="text", column=validation_text_format)
ecthra['validation']=validation

In [ ]:
ecthra=ecthra.remove_columns('labels')
ecthra=ecthra.rename_column('labels_one_hot', 'labels')

In [ ]:
from transformers import AutoTokenizer, YosoForSequenceClassification

In [ ]:
#add fine-tuning model


model_ckpt = "uw-madison/yoso-4096"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, problem_type="multi_label_classification")



In [ ]:
def tokenize_and_encode(examples):
  return tokenizer(examples["text"], truncation=True)

In [ ]:
cols = ecthra["train"].column_names
cols.remove("labels")
ds_enc = ecthra.map(tokenize_and_encode, batched=True, remove_columns=cols)


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
ds_enc['train']

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 9000
})

In [ ]:
len(ds_enc['train'][0]['input_ids'])

512

In [ ]:
ds_enc.set_format("torch")

In [ ]:
num_labels=10
model = YosoForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels, problem_type="multi_label_classification",ignore_mismatched_sizes=True)

Some weights of YosoForSequenceClassification were not initialized from the model checkpoint at uw-madison/yoso-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

In [ ]:
model.to('cuda')

YosoForSequenceClassification(
  (yoso): YosoModel(
    (embeddings): YosoEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): YosoEncoder(
      (layer): ModuleList(
        (0-11): 12 x YosoLayer(
          (attention): YosoAttention(
            (self): YosoSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): YosoSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [ ]:
# define the arguments
batch_size = 4
metric_name = "f1"

In [ ]:
args = TrainingArguments(
    f"output",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.05,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
# reference: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score,recall_score
from transformers import EvalPrediction
import sklearn.metrics as skm
import torch
import json


#function below returns the result of the model
def multi_label_metrics(predictions, labels, threshold=0.6):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    recall=recall_score(y_true, y_pred, average='macro')
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    #matrixes = skm.confusion_matrix(y_true, y_pred)
    cm = skm.multilabel_confusion_matrix(y_true, y_pred).tolist()
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'recall':recall,
               'matrixes':cm

               }
    return metrics

#function below is a helper function to call the final result
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [ ]:
#define trainer
trainer = Trainer(
    model=model, args=args,
    train_dataset=ds_enc["train"],
    eval_dataset=ds_enc["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [ ]:
trainer.evaluate()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer is attempting to log a value of "[[[943, 0], [57, 0]], [[807, 0], [193, 0]], [[813, 0], [187, 0]], [[700, 0], [300, 0]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[861, 0], [139, 0]]]" of type <class 'list'> for key "eval/matrixes" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.6896987557411194,
 'eval_f1': 0.0,
 'eval_roc_auc': 0.5,
 'eval_accuracy': 0.175,
 'eval_recall': 0.0,
 'eval_matrixes': [[[943, 0], [57, 0]],
  [[807, 0], [193, 0]],
  [[813, 0], [187, 0]],
  [[700, 0], [300, 0]],
  [[913, 0], [87, 0]],
  [[996, 0], [4, 0]],
  [[958, 0], [42, 0]],
  [[967, 0], [33, 0]],
  [[982, 0], [18, 0]],
  [[861, 0], [139, 0]]],
 'eval_runtime': 13.0402,
 'eval_samples_per_second': 76.686,
 'eval_steps_per_second': 19.171}

In [ ]:
import torch
torch.cuda.empty_cache()
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy,Recall,Matrixes
1,0.236800,0.279839,0.246753,0.571991,0.237000,0.057000,"[[[943, 0], [57, 0]], [[807, 0], [193, 0]], [[813, 0], [187, 0]], [[545, 155], [129, 171]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[861, 0], [139, 0]]]"
2,0.204300,0.239720,0.413622,0.645187,0.309000,0.159391,"[[[943, 0], [57, 0]], [[710, 97], [98, 95]], [[813, 0], [187, 0]], [[615, 85], [140, 160]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[822, 39], [60, 79]]]"
3,0.186900,0.218396,0.481527,0.674758,0.381000,0.202503,"[[[941, 2], [47, 10]], [[768, 39], [110, 83]], [[802, 11], [146, 41]], [[615, 85], [132, 168]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[825, 36], [50, 89]]]"
4,0.170900,0.214536,0.529680,0.706116,0.408000,0.253552,"[[[937, 6], [39, 18]], [[727, 80], [74, 119]], [[768, 45], [99, 88]], [[638, 62], [148, 152]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[826, 35], [52, 87]]]"
5,0.162100,0.205881,0.557681,0.716596,0.449000,0.260218,"[[[937, 6], [38, 19]], [[768, 39], [87, 106]], [[769, 44], [93, 94]], [[632, 68], [127, 173]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[834, 27], [50, 89]]]"


Trainer is attempting to log a value of "[[[943, 0], [57, 0]], [[807, 0], [193, 0]], [[813, 0], [187, 0]], [[545, 155], [129, 171]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[861, 0], [139, 0]]]" of type <class 'list'> for key "eval/matrixes" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[[943, 0], [57, 0]], [[710, 97], [98, 95]], [[813, 0], [187, 0]], [[615, 85], [140, 160]], [[913, 0], [87, 0]], [[996, 0], [4, 0]], [[958, 0], [42, 0]], [[967, 0], [33, 0]], [[982, 0], [18, 0]], [[822, 39], [60, 79]]]" of type <class 'list'> for key "eval/matrixes" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[[941, 2], [47, 10]], [[768, 39], [110, 83]], [[802, 11], [146, 41]], [[615, 85], [132, 168]], [[913, 0], [87, 0]], [[996, 0]

TrainOutput(global_step=11250, training_loss=0.2009485080295139, metrics={'train_runtime': 1386.3561, 'train_samples_per_second': 32.459, 'train_steps_per_second': 8.115, 'total_flos': 1.18377129969588e+16, 'train_loss': 0.2009485080295139, 'epoch': 5.0})

In [ ]:
x=trainer.evaluate()

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

confusion_matrices = x['eval_matrixes']


labels = ['Positive', 'Negative']


fig, axs = plt.subplots(2, 5, figsize=(15, 6))
for i, ax in enumerate(axs.flatten()):
    sns.heatmap(confusion_matrices[i], annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_title(f'Confusion Matrix {i+1}')
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('Actual Label')


plt.tight_layout()

plt.show()

